In [374]:
import pdfplumber
import re
import os
# os.chdir("..")
os.getcwd()

'd:\\Project\\invoice_pdf2csv'

In [375]:
invoice = {}

In [376]:
pdf_path = r"input\flipkart2.pdf"

In [377]:
flipkart_bboxes = {
    "invoice_number": (413, 80, 570, 100),
    "seller_details": (4, 22, 413, 100),
    "order_details": (4, 110, 143, 225),
    "billing_address": (144, 110, 279, 225),
    "shipping_address": (280, 110, 420, 225),
    "product_details": (4, 282, 100, 430),
    # "product_name": (102, 282, 260, 400),
    # "product_quantity": (262, 282, 294, 340),
    # "product_gross_amoount": (304, 282, 342, 340),
    # "product_discount_coupon": (350, 282, 398, 340),
    # "product_taxable_value": (410, 282, 468, 340),
    # "product_gst": (470, 282, 525, 340),
    # "product_total": (525, 282, 570, 340),
    # "shipping_gross_amoount": (304, 342, 342, 420),
    # "shipping_discount_coupon": (350, 342, 398, 420),
    # "shipping_taxable_value": (410, 342, 468, 420),
    # "shipping_gst": (470, 342, 525, 420),
    # "shipping_total": (525, 342, 570, 420),
    # "grand_total": (485, 472, 580, 500),
    # "gst_type": (470, 250, 525, 280),
    # "amount_table": (4, 230, 575, 460)
}

In [378]:
def debug_bbox(pdf_path, target_bboxes):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[0] # Assuming the data is on the first page

            # Create an image of the page
            im = page.to_image()

            colors = ["red", "blue", "green", "purple", "orange", "cyan", "magenta"]
            color_index = 0

            for name, bbox_coords in target_bboxes.items():
                color = colors[color_index % len(colors)] # Cycle through colors
                im.draw_rect(bbox_coords, stroke=color, stroke_width=2, fill=None) # fill=None means no fill
                print(f"Drawing '{name}' in {color}.")
                color_index += 1

            # Save the image to a file to inspect
            im.save("debug_bbox_target.png")

        print(f"Debug image 'debug_bbox_target.png' created")

    except FileNotFoundError:
        print(f"Error: The PDF file '{pdf_path}' was not found. Please check the path.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [379]:
debug_bbox(pdf_path, flipkart_bboxes)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Drawing 'invoice_number' in red.
Drawing 'seller_details' in blue.
Drawing 'order_details' in green.
Drawing 'billing_address' in purple.
Drawing 'shipping_address' in orange.
Drawing 'product_details' in cyan.
Debug image 'debug_bbox_target.png' created


In [380]:
def extract_text_from_bbox(pdf_path, bounding_box):
    """
    Extracts text from a specified bounding box within a PDF page.

    Args:
        pdf_path (str): The path to the PDF file.
        target_bbox (tuple): A tuple (x0, top, x1, bottom) defining the bounding box
                             from which to extract text. Coordinates are in PDF points.

    Returns:
        str: The extracted text from the specified bounding box, or None if no text is found.
    """
    extracted_text = None
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]  # Assuming data is on the first page

        # Use the extract_text() method with the bbox parameter
        cropped_page = page.crop(bounding_box)
        extracted_text = cropped_page.extract_text()
            
    return extracted_text

In [381]:
extracted_text_output = {}
for name, bbox in flipkart_bboxes.items():
    text = extract_text_from_bbox(pdf_path, bbox)
    # Store the extracted text using the same name (key) as the bbox
    extracted_text_output[name] = text.split("\n")

print("\n--- Content of extracted_text_output ---")
for key, value in extracted_text_output.items():
    print(f"{key}: '{value}'")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



--- Content of extracted_text_output ---
invoice_number: '['Invoice Number # EAAAAC-66692608']'
seller_details: '['Tax Invoice', 'Sold By: Jeeves consumer services Private Limited ,', 'Ship-from Address: L-169, 13th Cross, 5th Main,, Sector - 6, HSR Layout,, BANGALORE , KARNATAKA, 560102, IN-KA', 'GSTIN - 29AABCJ9421C1ZP']'
order_details: '['Order ID:', 'OD329896130732373100', 'Order Date: 13-12-2023', 'Invoice Date: 13-12-2023', 'PAN: AABCJ9421C']'
billing_address: '['Bill To', 'NIDHI Srivastava', 'MMMTU Gorakhpur, Deoriya', 'road Singhariya kunraghat', 'gorakhpur.', 'Gorakhpur 273016 Uttar', 'Pradesh', 'Phone: xxxxxxxxxx']'
shipping_address: '['Ship To', 'NIDHI Srivastava', 'MMMTU Gorakhpur, Deoriya', 'road Singhariya kunraghat', 'gorakhpur.', 'Gorakhpur 273016 Uttar', 'Pradesh', 'Phone: xxxxxxxxxx']'
product_details: '['Digital Voucher Code', 'FSN: DGVGFXDYJMCCH8P', 'HSN/SAC: 997331']'


In [382]:
invoice["invoice_number"] = re.search(r"Invoice Number #\s+([A-Z0-9]+)", extracted_text_output['invoice_number'][0]).group(1).strip()

In [383]:
extracted_text_output['seller_details'][1], " ".join(extracted_text_output['seller_details'][2:-1]), extracted_text_output['seller_details'][-1]

('Sold By: Jeeves consumer services Private Limited ,',
 'Ship-from Address: L-169, 13th Cross, 5th Main,, Sector - 6, HSR Layout,, BANGALORE , KARNATAKA, 560102, IN-KA',
 'GSTIN - 29AABCJ9421C1ZP')

In [384]:
invoice["seller_name"] = extracted_text_output['seller_details'][1][len("Sold By: "):-2]

In [385]:
invoice["seller_address"] = " ".join(extracted_text_output['seller_details'][2:-1])[len("Ship-from Address: "):-1].strip()

In [386]:
invoice["gst_number"] = extracted_text_output['seller_details'][-1][len("GSTIN - "):]

In [387]:
extracted_text_output['order_details']

['Order ID:',
 'OD329896130732373100',
 'Order Date: 13-12-2023',
 'Invoice Date: 13-12-2023',
 'PAN: AABCJ9421C']

In [388]:
invoice["order_id"] = extracted_text_output['order_details'][1].strip()
invoice["order_date"] = extracted_text_output['order_details'][2][len("Order Date: "):].strip()
invoice["invoice_date"] = extracted_text_output['order_details'][3][len("Invoice Date: "):].strip()
invoice["pan_number"] = extracted_text_output['order_details'][4][len("PAN: "):].strip()
# invoice["corporate_identification_number"] = extracted_text_output['order_details'][5][len("CIN: "):].strip()

In [389]:
extracted_text_output["billing_address"]

['Bill To',
 'NIDHI Srivastava',
 'MMMTU Gorakhpur, Deoriya',
 'road Singhariya kunraghat',
 'gorakhpur.',
 'Gorakhpur 273016 Uttar',
 'Pradesh',
 'Phone: xxxxxxxxxx']

In [390]:
invoice["bill_to_name"] = extracted_text_output["billing_address"][1].strip()
invoice["bill_to_address"] = " ".join(extracted_text_output["billing_address"][2:-1]).strip()
# invoice["bill_to_phone_number"] = extracted_text_output["billing_address"][-1][len("Phone: "):].strip()

In [391]:
extracted_text_output['shipping_address']

['Ship To',
 'NIDHI Srivastava',
 'MMMTU Gorakhpur, Deoriya',
 'road Singhariya kunraghat',
 'gorakhpur.',
 'Gorakhpur 273016 Uttar',
 'Pradesh',
 'Phone: xxxxxxxxxx']

In [392]:
invoice["ship_to_name"] = extracted_text_output["shipping_address"][1].strip()
invoice["ship_to_address"] = " ".join(extracted_text_output["shipping_address"][2:-1]).strip()
# invoice["ship_to_phone_number"] = extracted_text_output["shipping_address"][-1][len("Phone: "):].strip()

In [393]:
extracted_text_output["product_details"]

['Digital Voucher Code', 'FSN: DGVGFXDYJMCCH8P', 'HSN/SAC: 997331']

In [394]:
invoice["product_category"] = extracted_text_output["product_details"][0].strip()

In [395]:
import camelot

stream_tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream', suppress_stdout=False)
list_of_dfs = []
for i, table in enumerate(stream_tables):
    df = table.df.copy()
    list_of_dfs.append(df)

df = list_of_dfs[1]
df


CropBox missing from /Page, defaulting to MediaBox


,0,1,2,3,4,5,6
0,,Phone: xxxxxxxxxx,Phone: xxxxxxxxxx,,,,
1,Total items: 1,,,,,,
2,Product,Title,Qty\nGross,Discounts,Taxable,IGST,Total ₹
3,,,Amount ₹,/Coupons ₹,Value ₹,₹,
4,Digital Voucher Code,MICROSOFT 365 Personal,1\n1098.00,-1098.00,0.00,0.00,0.00
5,FSN: DGVGFXDYJMCCH8P6,6 Month subscription,,,,,
6,HSN/SAC: 997331,included,,,,,
7,,IGST:\n 18.0 %,,,,,


In [396]:
import numpy as np
import pandas as pd

#For some invoices columns may be merged. Seperating the columns for them.
columns_to_split = 5, 2, 1, 0
for target_column_name in columns_to_split:
        if target_column_name in df.columns:
            # 1. Split the target column by '\n'.
            split_parts = df[target_column_name].astype(str).str.split('\n', n=1, expand=True)

            # Check if split_parts has only one column (meaning no '\n' was found in any row)
            if split_parts.shape[1] == 1:
                # If only one column, it means all original values go to the 'right' column.
                new_col_left_values = np.full(len(df), np.nan)
                new_col_right_values = split_parts[0] 
            else:
                # If two columns, proceed as before (splitting occurred)
                # Identify rows where no newline was found (split_parts[1] will be NaN).
                no_split_mask = split_parts[1].isna()

                # 3. Prepare the new values for the 'left' (original column) and 'right' (new column).
                new_col_left_values = np.where(no_split_mask, np.nan, split_parts[0])
                new_col_right_values = np.where(no_split_mask, split_parts[0], split_parts[1])


            # Get the current list of column names and find the index of the target column.
            cols = df.columns.tolist()
            try:
                target_col_index = cols.index(target_column_name)
            except ValueError:
                print(f"Error: Column '{target_column_name}' not found in the DataFrame.")
                exit()

            # Insert the new column at the correct position.
            df.insert(loc=target_col_index + 1, column='TEMP_NEW_COL', value=new_col_right_values)

            # Assign the prepared 'left' values back to the original target column.
            df[target_column_name] = new_col_left_values

            # Rename the newly inserted column.
            renaming_map = {'TEMP_NEW_COL': target_column_name+1}

            for col_idx in range(target_col_index + 1, len(cols)):
                original_shifted_col_name = cols[col_idx]
                try:
                    new_shifted_col_name = int(original_shifted_col_name) + 1
                    renaming_map[original_shifted_col_name] = new_shifted_col_name
                except ValueError:
                    pass # Keeping non-numeric column names as they are

            df = df.rename(columns=renaming_map)

        else:
            print(f"Column '{target_column_name}' not found in the DataFrame. No transformation applied.")

In [397]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,NaN,,NaN,Phone: xxxxxxxxxx,NaN,Phone: xxxxxxxxxx,,,NaN,,
1,NaN,Total items: 1,NaN,,NaN,,,,NaN,,
2,NaN,Product,NaN,Title,Qty,Gross,Discounts,Taxable,NaN,IGST,Total ₹
3,NaN,,NaN,,NaN,Amount ₹,/Coupons ₹,Value ₹,NaN,₹,
4,NaN,Digital Voucher Code,NaN,MICROSOFT 365 Personal,1,1098.00,-1098.00,0.00,NaN,0.00,0.00
5,NaN,FSN: DGVGFXDYJMCCH8P6,NaN,6 Month subscription,NaN,,,,NaN,,
6,NaN,HSN/SAC: 997331,NaN,included,NaN,,,,NaN,,
7,NaN,,IGST:,18.0 %,NaN,,,,NaN,,


In [398]:
def index_finder(string_to_find, df):
    found_occurrences = []
    for column_name in df.columns:
        contains_string_mask = df[column_name].astype(str).str.contains(string_to_find, case=False, na=False)

        matching_row_indices_in_this_column = df.index[contains_string_mask].tolist()

        for row_idx in matching_row_indices_in_this_column:
            found_occurrences.append([row_idx, column_name])

    return found_occurrences

In [399]:
def product_name_extractor(df):
    # Looking for the colomn that contains "Title"
    # if index_finder("Title", df):
    match_indices = index_finder("Title", df)
    # elif index_finder("Shipping And Packaging Charges", df):
    #     match_indices = index_finder("Shipping And Packaging Charges", df)
    # Checking for the first occurance of ":" and removing everything below it including ":" row.
    title = df.iloc[4:, match_indices[0][1]]
    if title.astype(str).str.contains(":").any():
        column_match_found = title.astype(str).str.contains(":")
    elif title.astype(str).str.contains("%").any():
        column_match_found = title.astype(str).str.contains("%")
    index_to_start_removal = title[column_match_found].index[0]
    indices_to_keep = title.index[title.index < index_to_start_removal]
    product_name = " ".join(title.loc[indices_to_keep]).strip(" ")
    return product_name

In [400]:
invoice["product_name"] = product_name_extractor(df)

In [401]:
if df[index_finder("Qty", df)[0][1]].isnull().any():
  rows_to_add = int(df[df[index_finder("Qty", df)[0][1]].notnull()].index.to_list()[1]) - (int(index_finder("Qty", df)[0][0]))

In [402]:
if (df[index_finder("Qty", df)[0][1]].astype(str).str.strip() == '').any():
  rows_to_add = int(df[df[index_finder("Qty", df)[0][1]].astype(str).str.strip() != ''].index.to_list()[1]) - int(index_finder("Qty", df)[0][0])

In [403]:
rows_to_add

2

In [404]:
invoice["product_quantity"] = df.iloc[index_finder("Qty", df)[0][0]+ rows_to_add, index_finder("Qty", df)[0][1]]


In [405]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,NaN,,NaN,Phone: xxxxxxxxxx,NaN,Phone: xxxxxxxxxx,,,NaN,,
1,NaN,Total items: 1,NaN,,NaN,,,,NaN,,
2,NaN,Product,NaN,Title,Qty,Gross,Discounts,Taxable,NaN,IGST,Total ₹
3,NaN,,NaN,,NaN,Amount ₹,/Coupons ₹,Value ₹,NaN,₹,
4,NaN,Digital Voucher Code,NaN,MICROSOFT 365 Personal,1,1098.00,-1098.00,0.00,NaN,0.00,0.00
5,NaN,FSN: DGVGFXDYJMCCH8P6,NaN,6 Month subscription,NaN,,,,NaN,,
6,NaN,HSN/SAC: 997331,NaN,included,NaN,,,,NaN,,
7,NaN,,IGST:,18.0 %,NaN,,,,NaN,,


In [406]:
# invoice["product_discount_coupons"] = df.iloc[index_finder("Discounts", df)[0][0] + rows_to_add, index_finder("Discounts", df)[0][1]]
# invoice["shipping_discount_coupons"] = df.iloc[-1, index_finder("Discounts", df)[0][1]]

In [407]:
invoice["product_taxable_value"] = df.iloc[index_finder("Taxable", df)[0][0] + rows_to_add, index_finder("Taxable", df)[0][1]]
# invoice["shipping_taxable_value"] = df.iloc[-1, index_finder("Taxable", df)[0][1]]

In [408]:
invoice["product_total"] = df.iloc[index_finder("Total ₹", df)[0][0] + rows_to_add, index_finder("Total ₹", df)[0][1]]
# invoice["shipping_total"] = df.iloc[-1, index_finder("Total ₹", df)[0][1]]

In [409]:
if index_finder("SGST", df):
    invoice["GST"] = str(float(df.iloc[index_finder("SGST", df)[1][0] + rows_to_add, index_finder("SGST", df)[1][1]]) + float(df.iloc[index_finder("CGST", df)[1][0]+3, index_finder("CGST", df)[1][1]]))

In [410]:
if index_finder("IGST", df):
  invoice["GST"] = df.iloc[index_finder("IGST", df)[1][0] + rows_to_add, index_finder("IGST", df)[1][1]]


In [411]:
if index_finder("UGST", df):
  invoice["GST"] = df.iloc[index_finder("UGST", df)[1][0] + rows_to_add, index_finder("UGST", df)[1][1]]


In [412]:
# invoice["grand_total"] = str(float(invoice["product_total"]) + float(invoice["shipping_total"]))


In [413]:
invoice

{'invoice_number': 'EAAAAC',
 'seller_name': 'Jeeves consumer services Private Limited',
 'seller_address': 'L-169, 13th Cross, 5th Main,, Sector - 6, HSR Layout,, BANGALORE , KARNATAKA, 560102, IN-K',
 'gst_number': '29AABCJ9421C1ZP',
 'order_id': 'OD329896130732373100',
 'order_date': '13-12-2023',
 'invoice_date': '13-12-2023',
 'pan_number': 'AABCJ9421C',
 'bill_to_name': 'NIDHI Srivastava',
 'bill_to_address': 'MMMTU Gorakhpur, Deoriya road Singhariya kunraghat gorakhpur. Gorakhpur 273016 Uttar Pradesh',
 'ship_to_name': 'NIDHI Srivastava',
 'ship_to_address': 'MMMTU Gorakhpur, Deoriya road Singhariya kunraghat gorakhpur. Gorakhpur 273016 Uttar Pradesh',
 'product_category': 'Digital Voucher Code',
 'product_name': 'MICROSOFT 365 Personal 6 Month subscription included',
 'product_quantity': '1',
 'product_taxable_value': '0.00',
 'product_total': '0.00',
 'GST': '0.00'}